In [101]:
import ray

In [102]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2025-09-05 16:25:34,907	INFO worker.py:1942 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.3
Ray version:,2.49.1
Dashboard:,http://127.0.0.1:8265


In [103]:
ray.cluster_resources()

{'node:172.16.208.159': 1.0,
 'node:__internal_head__': 1.0,
 'object_store_memory': 1414835404.0,
 'CPU': 12.0,
 'memory': 3301282612.0}

In [104]:
import pandas as pd

In [105]:
# Data Ingestion
DATASET_LOC = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/dataset.csv"
df=pd.read_csv(DATASET_LOC)
df.head()

,id,created_on,title,description,tag
0,6,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...,computer-vision
1,7,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...,computer-vision
2,9,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla...",other
3,15,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...,other
4,25,2020-03-07 23:04:31,AttentionWalk,"A PyTorch Implementation of ""Watch Your Step: ...",other


In [106]:
from sklearn.model_selection import train_test_split

In [107]:
# df.tag.value_counts()

In [108]:
# split dataset
# test_size=0.2
# train_df,val_df=train_test_split(df,test_size=test_size,random_state=1234)

In [109]:
# train_df.tag.value_counts()

In [110]:
# val_df.tag.value_counts() * int((1-test_size)/test_size)

In [111]:
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns;sns.set_theme()
import warnings; warnings.filterwarnings("ignore")
from wordcloud import WordCloud, STOPWORDS

In [112]:
# all_tags=Counter(df.tag)
# all_tags.most_common()

In [113]:
# tags, tag_counts = zip(*all_tags.most_common())
# plt.figure(figsize=(10, 3))
# ax = sns.barplot(x=list(tags), y=list(tag_counts))
# ax.set_xticklabels(tags, rotation=0, fontsize=8)
# plt.title("Tag distribution", fontsize=14)
# plt.ylabel("# of projects", fontsize=12)
# plt.show()

In [114]:
# Most frequent tokens for each tag
# tag = "natural-language-processing"
# plt.figure(figsize=(10, 3))
# subset = df[df.tag == tag]
# text = subset.title.values
# cloud = WordCloud(
#     stopwords=STOPWORDS,
#     background_color="black",
#     collocations=False,
#     width=500,
#     height=300,
# ).generate(" ".join(text))
# plt.axis("off")
# plt.imshow(cloud)

In [115]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [116]:
nltk.download("stopwords")
STOPWORDS=stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bs00927/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
def clean_text(text, stopwords=STOPWORDS):
    """Clean raw text string."""
    # Lower
    text = text.lower()

    # Remove stopwords
    pattern = re.compile(r"\b(" + r"|".join(stopwords) + r")\b\s*")
    text = pattern.sub("", text)

    # Spacing and filters
    text = re.sub(
        r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text
    )  # add spacing
    text = re.sub("[^A-Za-z0-9]+", " ", text)  # remove non alphanumeric chars
    text = re.sub(" +", " ", text)  # remove multiple spaces
    text = text.strip()  # strip white space at the ends
    text = re.sub(r"http\S+", "", text)  #  remove links

    return text

In [118]:
# def decode(indices,index_to_class):
#     return [index_to_class[i] for i in indices]

In [119]:
# index_to_class={v:k for k,v in class_to_index.items()}
# decode(df.head()["tag"].values,index_to_class)

In [120]:
import numpy as np
from transformers import BertTokenizer

In [121]:
def tokenize(batch):
    tokenizer = BertTokenizer.from_pretrained(
        "allenai/scibert_scivocab_uncased", return_dict=False
    )
    encoded_inputs = tokenizer(batch["text"].tolist(), return_tensors="np", padding="longest")
    return dict(ids=encoded_inputs["input_ids"], mask=encoded_inputs["attention_mask"], targets=np.array(batch["tag"]))

In [122]:
def preprocess(df,class_to_index):
    """Preprocess the data."""
    df["text"] = df.title + " " + df.description  # feature engineering
    df["text"] = df.text.apply(clean_text)  # clean text
    df = df.drop(
        columns=["id", "created_on", "title", "description"], errors="ignore"
    )  # clean dataframe
    df = df[["text", "tag"]]  # rearrange columns
    df["tag"] = df["tag"].map(class_to_index)  # label encoding
    outputs = tokenize(df)
    return outputs

In [123]:
# preprocess(train_df,class_to_index)

In [124]:
ds = ray.data.read_csv(DATASET_LOC)
ds=ds.random_shuffle(seed=1234)
ds.take(1)

2025-09-05 16:25:37,609	INFO logging.py:295 -- Registered dataset logger for dataset dataset_2_0
2025-09-05 16:25:37,615	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_2_0. Full logs are in /tmp/ray/session_2025-09-05_16-25-32_594366_316544/logs/ray-data
2025-09-05 16:25:37,615	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_2_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> LimitOperator[limit=1]
Running 0: 0.00 row [00:00, ? row/s]



















Running Dataset: dataset_2_0. Active & requested resources: 1/12 CPU, 256.0MB/674.6MB object store: : 0.00 row [00:01, ? row/s]

























Running Dataset: dataset_2_0. Active & requested resources: 0/12 CPU, 138.4KB/674.6MB object store: : 0.00 row [00:03, ? row/s]2025-09-05 16:25:40,976	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_2_0 execution finished in 3.36 seconds
                                                      

[{'id': 695,
  'created_on': datetime.datetime(2020, 5, 2, 21, 33, 31),
  'title': 'Five Cool Python Libraries for Data Science',
  'description': 'Python is a best friend for the majority of the Data Scientists. Libraries make their life simpler. I have come across five cool Python libraries while working ',
  'tag': 'natural-language-processing'}]

In [125]:
from ray.data import Dataset
from typing import Dict, List, Tuple

In [126]:
def stratify_split(
    ds: Dataset,
    stratify: str,
    test_size: float,
    shuffle: bool = True,
    seed: int = 1234,
) -> Tuple[Dataset, Dataset]:
    """Split a dataset into train and test splits with equal
    amounts of data points from each class in the column we
    want to stratify on.

    Args:
        ds (Dataset): Input dataset to split.
        stratify (str): Name of column to split on.
        test_size (float): Proportion of dataset to split for test set.
        shuffle (bool, optional): whether to shuffle the dataset. Defaults to True.
        seed (int, optional): seed for shuffling. Defaults to 1234.

    Returns:
        Tuple[Dataset, Dataset]: the stratified train and test datasets.
    """

    def _add_split(
        df: pd.DataFrame,
    ) -> pd.DataFrame:  # pragma: no cover, used in parent function
        """Naively split a dataframe into train and test splits.
        Add a column specifying whether it's the train or test split."""
        train, test = train_test_split(
            df, test_size=test_size, shuffle=shuffle, random_state=seed
        )
        train["_split"] = "train"
        test["_split"] = "test"
        return pd.concat([train, test])

    def _filter_split(
        df: pd.DataFrame, split: str
    ) -> pd.DataFrame:  # pragma: no cover, used in parent function
        """Filter by data points that match the split column's value
        and return the dataframe with the _split column dropped."""
        return df[df["_split"] == split].drop("_split", axis=1)

    # Train, test split with stratify
    grouped = ds.groupby(stratify).map_groups(
        _add_split, batch_format="pandas"
    )  # group by each unique value in the column we want to stratify on
    train_ds = grouped.map_batches(
        _filter_split, fn_kwargs={"split": "train"}, batch_format="pandas"
    )  # combine
    test_ds = grouped.map_batches(
        _filter_split, fn_kwargs={"split": "test"}, batch_format="pandas"
    )  # combine

    # Shuffle each split (required)
    train_ds = train_ds.random_shuffle(seed=seed)
    test_ds = test_ds.random_shuffle(seed=seed)

    return train_ds, test_ds

In [127]:
test_size=0.2
train_ds,val_ds = stratify_split(ds,stratify="tag",test_size=test_size)

In [ ]:
tags=train_ds.unique(column="tag")
# print(tags)
class_to_index={tag:i for i,tag in enumerate(tags)}

2025-09-05 16:25:41,100	INFO logging.py:295 -- Registered dataset logger for dataset dataset_11_0
2025-09-05 16:25:41,108	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_11_0. Full logs are in /tmp/ray/session_2025-09-05_16-25-32_594366_316544/logs/ray-data
2025-09-05 16:25:41,109	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_11_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]
Running 0: 0.00 row [00:00, ? row/s]





































































































































































































Running Dataset: dataset_11_0. Active & requested resources: 1/12 CPU, 256.0MB/674.6MB object store: : 0.00 row [00:01, ? row/s

Running Dataset: dataset_13_0. Active & requested resources: 1/12 CPU, 16.9KB/674.6MB object store: : 0.00 row [00:01, ? row/s]



































































































































































2025-09-05 16:25:46,263	WARNING streaming_executor_state.py:793 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: id: int64
created_on: timestamp[s]
title: string
description: string
tag: string, new schema: . This may lead to unexpected behavior.


































































































































































Running Dataset: dataset_13_0. Active & requested resources: 1/12 CPU, 256.2MB/674.6MB object store: : 0.00 row [00:02, ? row/s]



































Running Dataset: dataset_13_0. Active & requested resources: 1/12 CPU, 256.2MB/674.6MB o

In [129]:
simple_ds=train_ds.map_batches(
    preprocess,
    fn_kwargs={"class_to_index":class_to_index},
    batch_format="pandas"
)
simple_ds.show(1)

2025-09-05 16:25:44,706	INFO logging.py:295 -- Registered dataset logger for dataset dataset_13_0
2025-09-05 16:25:44,714	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_13_0. Full logs are in /tmp/ray/session_2025-09-05_16-25-32_594366_316544/logs/ray-data
2025-09-05 16:25:44,715	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_13_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> TaskPoolMapOperator[MapBatches(preprocess)] -> LimitOperator[limit=1]
Running 0: 0.00 row [00:00, ? row/s]













































































2025-09-05 16:25:49,313	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_13_0 execution finished in 4.60 seconds
                                                                                                                 

{'ids': array([  102,   638, 30108,   453, 18205,  4631,  1904,   453, 18205,
        5560,  4631,  1904,  6283,  2117,  6773, 25644,  8855,  1519,
        1861,   103,     0,     0,     0,     0,     0,     0,     0,
           0,     0]), 'mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0]), 'targets': 3}
